# Applications Weighted Grading Automation

The main objecctive of this code is to automate the process of calculating the overall weighted grade of students' responses, from the weights and grades given to each question from the graded questions.